In [1]:
import pickle
import numpy as np
import tensorflow as tf

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [3]:
#load the CIFAR-10
def load_CIFAR10(pos, n_chunks=1):
    Xtr = []
    Ytr = []
    for i in range(n_chunks):
        train = unpickle(pos + '/data_batch_{0}'.format(i + 1))
        Xtr.extend(train[b'data'])
        Ytr.extend(train[b'labels'])
        test = unpickle(pos + '/test_batch')
        Xte = test[b'data']
        Yte = test[b'labels']
    return np.array(Xtr), np.array(Ytr), np.array(Xte), np.array(Yte)

In [4]:
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.zeros((Ytr.size, 10))
    Yte_onehot = np.zeros((Yte.size, 10))
    for i in range(Ytr.size):
        Ytr_onehot[i][Ytr[i]] = 1
    for i in range(Yte.size):
        Yte_onehot[i][Yte[i]] = 1
    return Ytr_onehot, Yte_onehot

In [5]:
# load the train and test data
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar-10-batches-py', 5)
                                 
# image data, each data size is 32*32*3
Xtr = Xtr.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("float")
Xte= Xte.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")

# label data of train and test data, label data is represented by one-hot encoding
Ytr_onehot, Yte_onehot = onehot_encoding(Ytr, Yte)


In [6]:
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

In [7]:
### Implement the layers of CNNs ###

In [8]:
W1 = tf.Variable(tf.random_normal([3,3,3,32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding="SAME")

batch_mean, batch_var = tf.nn.moments(L1,[0])
scale = tf.Variable(tf.ones([32,32,32]))
beta = tf.Variable(tf.zeros([32,32,32]))
L1 = tf.nn.batch_normalization(L1, batch_mean, batch_var,beta, scale, 1e-3)

L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides = [1,2,2,1], padding = "SAME")
L1 = tf.nn.dropout(L1, dropout_prob)

In [9]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = "SAME")

batch_mean, batch_var = tf.nn.moments(L2,[0])
scale = tf.Variable(tf.ones([16,16,64]))
beta = tf.Variable(tf.zeros([16,16,64]))
L2 = tf.nn.batch_normalization(L2, batch_mean, batch_var,beta, scale, 1e-3)

L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding="SAME")
# L2 = tf.nn.dropout(L2, dropout_prob)

In [10]:
W3= tf.Variable(tf.random_normal([3,3,64,128], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides = [1,1,1,1], padding = "SAME")

batch_mean, batch_var = tf.nn.moments(L3,[0])
scale = tf.Variable(tf.ones([8,8,128]))
beta = tf.Variable(tf.zeros([8,8,128]))
L3 = tf.nn.batch_normalization(L3, batch_mean, batch_var,beta, scale, 1e-3)

L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, dropout_prob)

In [11]:
W4= tf.Variable(tf.random_normal([3,3,128,256], stddev=0.01))
L4 = tf.nn.conv2d(L3, W4, strides = [1,1,1,1], padding = "SAME")

batch_mean, batch_var = tf.nn.moments(L4,[0])
scale = tf.Variable(tf.ones([8,8,256]))
beta = tf.Variable(tf.zeros([8,8,256]))
L4 = tf.nn.batch_normalization(L4, batch_mean, batch_var,beta, scale, 1e-3)

L4 = tf.nn.relu(L4)
# L4 = tf.nn.dropout(L4, dropout_prob)

In [12]:
W5= tf.Variable(tf.random_normal([3,3,256,512], stddev=0.01))
L5 = tf.nn.conv2d(L4, W5, strides = [1,1,1,1], padding = "SAME")

batch_mean, batch_var = tf.nn.moments(L5,[0])
scale = tf.Variable(tf.ones([8,8,512]))
beta = tf.Variable(tf.zeros([8,8,512]))
L5 = tf.nn.batch_normalization(L5, batch_mean, batch_var,beta, scale, 1e-3)

L5 = tf.nn.relu(L5)
L5 = tf.nn.dropout(L5, dropout_prob)

In [13]:
#512
W6 = tf.Variable(tf.random_normal([8*8*512, 512], stddev = 0.01))
L6 = tf.reshape(L5, [-1, 8*8*512])
L6 = tf.matmul(L6, W6)

batch_mean, batch_var = tf.nn.moments(L6,[0])
scale = tf.Variable(tf.ones([512]))
beta = tf.Variable(tf.zeros([512]))
L6 = tf.nn.batch_normalization(L6, batch_mean, batch_var,beta, scale, 1e-3)

L6 = tf.nn.relu(L6)
#dropout
L6 = tf.nn.dropout(L6, dropout_prob)

In [14]:
W7 = tf.Variable(tf.random_normal([512,10],stddev = 0.01))
model = tf.matmul(L6, W7)

#### Cost and optimizer
***

In [15]:
# 0.002

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.002).minimize(cost)
# optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
# optimizer = tf.train.AdadeltaOptimizer(0.001).minimize(cost)

In [16]:
# initialize the variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [17]:
### Implement the train process ###

In [18]:
len(Xtr)

50000

In [19]:
batch_size = 400
total_batch = int(len(Xtr) / batch_size)

In [20]:
#dropout_prob:0.6
#range : 35

for epoch in range(50):
    total_cost=0
    for i in range(total_batch):
        batch_xs, batch_ys = Xtr[i*batch_size:(i+1)*batch_size], Ytr_onehot[i*batch_size:(i+1)*batch_size]
        #batch_xs = batch_xs.reshape(-1, 32, 32, 3)
        _, curr_loss = sess.run([optimizer, cost],feed_dict={X:batch_xs, 
                                                             Y:batch_ys,
                                                            dropout_prob: 0.6})
        total_cost += curr_loss
        
    print("Epoch:", "%04d" % (epoch +1),"Avg.cost = ", "{:,.3f}".format(total_cost/total_batch))
    correctness = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
    print('Accuracy', sess.run(accuracy, feed_dict = {X:Xte,
                                                      Y:Yte_onehot,
                                                      dropout_prob: 1.0}))

Epoch: 0001 Avg.cost =  1.312
Accuracy 0.656
Epoch: 0002 Avg.cost =  0.953
Accuracy 0.7186
Epoch: 0003 Avg.cost =  0.807
Accuracy 0.7557
Epoch: 0004 Avg.cost =  0.707
Accuracy 0.7748
Epoch: 0005 Avg.cost =  0.642
Accuracy 0.7851
Epoch: 0006 Avg.cost =  0.591
Accuracy 0.7903
Epoch: 0007 Avg.cost =  0.549
Accuracy 0.7919
Epoch: 0008 Avg.cost =  0.508
Accuracy 0.798
Epoch: 0009 Avg.cost =  0.468
Accuracy 0.8043
Epoch: 0010 Avg.cost =  0.437
Accuracy 0.8065
Epoch: 0011 Avg.cost =  0.403
Accuracy 0.8121
Epoch: 0012 Avg.cost =  0.378
Accuracy 0.8153
Epoch: 0013 Avg.cost =  0.360
Accuracy 0.8252
Epoch: 0014 Avg.cost =  0.329
Accuracy 0.8119
Epoch: 0015 Avg.cost =  0.315
Accuracy 0.8137
Epoch: 0016 Avg.cost =  0.294
Accuracy 0.8172
Epoch: 0017 Avg.cost =  0.268
Accuracy 0.8232
Epoch: 0018 Avg.cost =  0.256
Accuracy 0.823
Epoch: 0019 Avg.cost =  0.241
Accuracy 0.8192
Epoch: 0020 Avg.cost =  0.233
Accuracy 0.819
Epoch: 0021 Avg.cost =  0.220
Accuracy 0.8168
Epoch: 0022 Avg.cost =  0.209
Accuracy

In [21]:
### Implement the test process ###

In [22]:
correctness = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy, feed_dict = {X:Xte,
                                                  Y:Yte_onehot,
                                                  dropout_prob: 1.0}))

Accuracy 0.8345
